In [756]:
import pandas as pd
from sklearn.metrics import mean_squared_error

## Dataset File

In [757]:
dataset_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/train.csv?raw=true'
df = pd.read_csv('train.csv')

In [758]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [759]:
df.shape

(188533, 13)

In [760]:
df.drop(columns=['id', 'ext_col', 'int_col'], inplace=True)

In [761]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   brand         188533 non-null  object
 1   model         188533 non-null  object
 2   model_year    188533 non-null  int64 
 3   milage        188533 non-null  int64 
 4   fuel_type     183450 non-null  object
 5   engine        188533 non-null  object
 6   transmission  188533 non-null  object
 7   accident      186081 non-null  object
 8   clean_title   167114 non-null  object
 9   price         188533 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 14.4+ MB


In [762]:
df.isnull().sum()

,0
brand,0
model,0
model_year,0
milage,0
fuel_type,5083
engine,0
transmission,0
accident,2452
clean_title,21419
price,0


In [763]:
from datetime import datetime

current_year = datetime.now().year
df['car_age'] = current_year - df['model_year']

In [764]:
df['clean_title'] = df['clean_title'].map({'Yes': 0, 'Nan': 1})
df['clean_title'].fillna(df['clean_title'].median(), inplace=True)

In [765]:
df['accident'] = df['accident'].map({'None reported': 0, 'At least 1 accident or damage reported': 1})
df['accident'].fillna(df['accident'].median(), inplace=True)

In [766]:
df['fuel_type'] = df['fuel_type'].map({'Gasoline': 0, 'Diesel': 1, 'E85 Flex Fuel': 2, 'Hybrid': 3, 'Plug-In Hybrid': 3, 'not supported': 4})
df['fuel_type'].fillna(df['fuel_type'].median(), inplace=True)

In [767]:
df.loc[df['transmission'].str.contains('Automatic|A/T|AT', na=False), 'transmission'] = 'A/T'
df.loc[df['transmission'].str.contains('Manual|M/T|Mt', na=False), 'transmission'] = 'M/T'
df.loc[df['transmission'].str.contains('Manual|M/T|Mt', na=False), 'transmission'] = 'M/T'

In [768]:
df = df[df['transmission'] != 'SCHEDULED FOR OR IN PRODUCTION']
df = df[df['transmission'] != 'Single-Speed Fixed Gear']
other_values = [
    'Transmission Overdrive Switch',
    '–',
    '7-Speed',
    'F',
    '6-Speed',
    '2',
    'Variable',
    'CVT-F',
    'Single-Speed Fixed Gear'
]

# Replace specified values with 'Other'
df['transmission'] = df['transmission'].replace(other_values, 'Others')

In [769]:
df['HP'] = df['engine'].str.extract(r'(\d+\.?\d*)(?=HP)').astype(float)
df['Liters'] = df['engine'].str.extract(r'(\d+\.?\d*)(?=L)').astype(float)

In [770]:
df['HP'].fillna(df['HP'].median(), inplace=True)
df['Liters'].fillna(df['Liters'].median(), inplace=True)

In [771]:
df.drop(columns=['engine'], inplace=True)

In [772]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['transmission'] = label_encoder.fit_transform(df['transmission'])
df['transmission'].fillna(df['transmission'].median(), inplace=True)
df['brand'] = label_encoder.fit_transform(df['brand'])

In [773]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 188511 entries, 0 to 188532
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   brand         188511 non-null  int64  
 1   model         188511 non-null  object 
 2   model_year    188511 non-null  int64  
 3   milage        188511 non-null  int64  
 4   fuel_type     188511 non-null  float64
 5   transmission  188511 non-null  int64  
 6   accident      188511 non-null  float64
 7   clean_title   188511 non-null  float64
 8   price         188511 non-null  int64  
 9   car_age       188511 non-null  int64  
 10  HP            188511 non-null  float64
 11  Liters        188511 non-null  float64
dtypes: float64(5), int64(6), object(1)
memory usage: 18.7+ MB


In [774]:
df.head()

,brand,model,model_year,milage,fuel_type,transmission,accident,clean_title,price,car_age,HP,Liters
0,31,Cooper S Base,2007,213000,0.0,0,0.0,0.0,4200,17,172.0,1.6
1,28,LS V8,2002,143250,0.0,0,1.0,0.0,4999,22,252.0,3.9
2,9,Silverado 2500 LT,2002,136731,2.0,0,0.0,0.0,13900,22,320.0,5.3
3,16,G90 5.0 Ultimate,2017,19500,0.0,4,0.0,0.0,45000,7,420.0,5.0
4,36,Metris Base,2021,7388,0.0,0,0.0,0.0,97500,3,208.0,2.0


In [775]:
df.drop(columns=['model'], inplace=True)

In [776]:
df.head()

,brand,model_year,milage,fuel_type,transmission,accident,clean_title,price,car_age,HP,Liters
0,31,2007,213000,0.0,0,0.0,0.0,4200,17,172.0,1.6
1,28,2002,143250,0.0,0,1.0,0.0,4999,22,252.0,3.9
2,9,2002,136731,2.0,0,0.0,0.0,13900,22,320.0,5.3
3,16,2017,19500,0.0,4,0.0,0.0,45000,7,420.0,5.0
4,36,2021,7388,0.0,0,0.0,0.0,97500,3,208.0,2.0


In [777]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 188511 entries, 0 to 188532
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   brand         188511 non-null  int64  
 1   model_year    188511 non-null  int64  
 2   milage        188511 non-null  int64  
 3   fuel_type     188511 non-null  float64
 4   transmission  188511 non-null  int64  
 5   accident      188511 non-null  float64
 6   clean_title   188511 non-null  float64
 7   price         188511 non-null  int64  
 8   car_age       188511 non-null  int64  
 9   HP            188511 non-null  float64
 10  Liters        188511 non-null  float64
dtypes: float64(5), int64(6)
memory usage: 17.3 MB


In [778]:
!pip install scikit-learn # install scikit-learn if it is not already installed
from sklearn.model_selection import train_test_split # import train_test_split from sklearn.model_selection

X = df.drop(['price'], axis=1)
y = df['price']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [779]:
model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mean_squared_error(y_test, y_pred, squared=False)

69129.0525662546

## Test File

In [780]:
test_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/test.csv?raw=true'
dt=pd.read_csv('test.csv')

In [781]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            125690 non-null  int64 
 1   brand         125690 non-null  object
 2   model         125690 non-null  object
 3   model_year    125690 non-null  int64 
 4   milage        125690 non-null  int64 
 5   fuel_type     122307 non-null  object
 6   engine        125690 non-null  object
 7   transmission  125690 non-null  object
 8   ext_col       125690 non-null  object
 9   int_col       125690 non-null  object
 10  accident      124058 non-null  object
 11  clean_title   111451 non-null  object
dtypes: int64(3), object(9)
memory usage: 11.5+ MB


In [782]:
from datetime import datetime

current_year = datetime.now().year
dt['car_age'] = current_year - dt['model_year']

In [783]:
dt['fuel_type'] = dt['fuel_type'].map({'Gasoline': 0, 'Diesel': 1, 'E85 Flex Fuel': 2, 'Hybrid': 3, 'Plug-In Hybrid': 3, 'not supported': 4})
dt['fuel_type'].fillna(dt['fuel_type'].median(), inplace=True)

In [784]:
dt['HP'] = dt['engine'].str.extract(r'(\d+\.?\d*)(?=HP)').astype(float)
dt['Liters'] = dt['engine'].str.extract(r'(\d+\.?\d*)(?=L)').astype(float)

In [785]:
dt.drop(columns=['engine', 'ext_col', 'id', 'int_col', 'model'], inplace = True)

## Sample Submission File

In [786]:
sample_submission_url = 'https://github.com/robitussin/CCMACLRL_EXAM/blob/a46a4e2a001dedaefc9b431d480b508ce86c2d96/datasets/sample_submission.csv?raw=true'

sf=pd.read_csv('sample_submission.csv')

In [787]:
df.head()

,brand,model_year,milage,fuel_type,transmission,accident,clean_title,price,car_age,HP,Liters
0,31,2007,213000,0.0,0,0.0,0.0,4200,17,172.0,1.6
1,28,2002,143250,0.0,0,1.0,0.0,4999,22,252.0,3.9
2,9,2002,136731,2.0,0,0.0,0.0,13900,22,320.0,5.3
3,16,2017,19500,0.0,4,0.0,0.0,45000,7,420.0,5.0
4,36,2021,7388,0.0,0,0.0,0.0,97500,3,208.0,2.0


In [788]:
dt['transmission'] = label_encoder.fit_transform(dt['transmission'])
dt['transmission'].fillna(dt['transmission'].median(), inplace=True)
dt['brand'] = label_encoder.fit_transform(dt['brand'])

dt['accident'] = dt['accident'].map({'None reported': 0, 'At least 1 accident or damage reported': 1})
dt['accident'].fillna(dt['accident'].median(), inplace=True)

dt['clean_title'] = dt['clean_title'].map({'Yes': 0, 'Nan': 1})
dt['clean_title'].fillna(dt['clean_title'].median(), inplace=True)

dt['HP'].fillna(dt['HP'].median(), inplace=True)
dt['Liters'].fillna(dt['Liters'].median(), inplace=True)

In [789]:
sf = pd.read_csv('sample_submission.csv')
id = sf.pop('id')
y_pred = model.predict(dt)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': id,
    'class': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('results.csv', index=False)
print("Submission file created: submission_file.csv")

Submission file created: submission_file.csv
